# Preparing cashflow features for an training a clustered XGBoost model

### Project members:
- Marlene Ibrus
- Maare Karmen Oras
- Aleksandr Volžinski

In this file...

## I. Imports, User Configuration and Loading Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import joblib
import math
from k_means_constrained import KMeansConstrained
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna

In [2]:
USE_GPU = True # set False if no GPU
DEVICE = 'cuda' if USE_GPU else 'cpu'
TREE_METHOD = 'hist' if USE_GPU else 'hist'
OPTUNA_TRIALS = 40 # set higher (100-200) for better tuning
GLOBAL_CAP_PCT = 99.5 # cap target at this percentile during training
CLUSTERS = 5 # number of customer clusters
MIN_HISTORY_DAYS = 60 # minimum history for a customer to be modeled
OUTPUT_DIR = './models'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
df_customer = pd.read_csv('../Data/synthetic_sme_customers_processed.csv')

In [4]:
df_transaction = pd.read_csv('../Data/synthetic_sme_transactions_processed.csv')

## II. Create daily `net_flow` per Customer

In [5]:
df = df_transaction.copy()
df['BookingDatetime'] = pd.to_datetime(df['BookingDatetime'], errors='coerce')

In [6]:
# signed flow
df['flow'] = np.where(df['D_C'] == 'D', df['Amount_EUR'], -df['Amount_EUR'])

In [7]:
# aggregate to daily net_flow
daily_cashflow = (
df.groupby(['cust_id', pd.Grouper(key='BookingDatetime', freq='D')])['flow']
.sum()
.reset_index()
.rename(columns={'BookingDatetime':'date','flow':'net_flow'})
)

In [8]:
# Reindex per customer (fill missing days with 0)
full_rows = []
for cust, g in daily_cashflow.groupby('cust_id'):
    g = g.sort_values('date').set_index('date')
    idx = pd.date_range(g.index.min(), g.index.max(), freq='D')
    g2 = g.reindex(idx).rename_axis('date').reset_index()
    g2['cust_id'] = cust
    g2['net_flow'] = g2['net_flow'].fillna(0.0)
    full_rows.append(g2)

In [9]:
full_daily = pd.concat(full_rows, ignore_index=True)
print(f"Full daily shape: {full_daily.shape}")

Full daily shape: (682967, 3)


In [10]:
full_daily.head()

,date,cust_id,net_flow
0,2023-01-01,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,-0.020048
1,2023-01-02,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000
2,2023-01-03,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000
3,2023-01-04,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000
4,2023-01-05,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000


## III. Create 7-day target

In [11]:
full_daily = full_daily.sort_values(['cust_id','date']).reset_index(drop=True)
full_daily['target_7d'] = (
    full_daily.groupby('cust_id')['net_flow']
        .transform(lambda x: x.rolling(window=7, min_periods=7).sum().shift(-7))
)

In [12]:
# drop rows where target is NaN (last up-to-7 days)
full_daily = full_daily[~full_daily['target_7d'].isna()].reset_index(drop=True)
print(f"After target creation: {full_daily.shape}")

After target creation: (676261, 4)


In [13]:
full_daily.head()

,date,cust_id,net_flow,target_7d
0,2023-01-01,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,-0.020048,2.00481
1,2023-01-02,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481
2,2023-01-03,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481
3,2023-01-04,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481
4,2023-01-05,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481


## IV. Feature Engineering

In [18]:
# rolling windows
windows = [7,14,30,90]
for w in windows:
    full_daily[f'roll_sum_{w}'] = full_daily.groupby('cust_id')['net_flow'].transform(lambda x: x.rolling(window=w, min_periods=1).sum())
    full_daily[f'roll_mean_{w}'] = full_daily.groupby('cust_id')['net_flow'].transform(lambda x: x.rolling(window=w, min_periods=1).mean())
    full_daily[f'roll_std_{w}'] = full_daily.groupby('cust_id')['net_flow'].transform(lambda x: x.rolling(window=w, min_periods=1).std().fillna(0))
    full_daily[f'roll_count_nz_{w}'] = full_daily.groupby('cust_id')['net_flow'].transform(lambda x: x.rolling(window=w, min_periods=1).apply(lambda s: np.sum(s!=0)))

In [57]:
full_daily.head()

,date,cust_id,net_flow,target_7d,roll_sum_7,roll_mean_7,roll_std_7,roll_count_nz_7,roll_sum_14,roll_mean_14,...,std_flow_y,total_flow_y,n_days_y,max_flow_y,active_days_90_y,customer_type,parent_company_flag,CUST_GRP_ID,language,age
0,2023-01-01,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,-0.020048,2.00481,-0.020048,-0.020048,0.000000,1.0,-0.020048,-0.020048,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
1,2023-01-02,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.010024,0.014176,1.0,-0.020048,-0.010024,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
2,2023-01-03,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.006683,0.011575,1.0,-0.020048,-0.006683,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
3,2023-01-04,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.005012,0.010024,1.0,-0.020048,-0.005012,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
4,2023-01-05,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.004010,0.008966,1.0,-0.020048,-0.004010,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1


In [58]:
# lags and baseline
full_daily["lag_1"] = (
    full_daily.groupby("cust_id")["net_flow"]
    .shift(1)
    .fillna(0)
)

# Baseline: previous 7-day sum
full_daily["lag_7_sum"] = (
    full_daily.groupby("cust_id")["net_flow"]
    .transform(lambda x: x.shift(1).rolling(7, min_periods=1).sum())
    .fillna(0)
)

In [59]:
full_daily['dayofweek'] = full_daily['date'].dt.dayofweek
full_daily['is_weekend'] = full_daily['dayofweek'].isin([5,6]).astype(int)
full_daily['day'] = full_daily['date'].dt.day
full_daily['month'] = full_daily['date'].dt.month
full_daily['year'] = full_daily['date'].dt.year

In [60]:
# quick activity features per customer (aggregation)
agg = (
    full_daily.groupby("cust_id")["net_flow"]
    .agg(['mean','std','sum','count','max'])
    .rename(columns={
        'mean':'mean_flow',
        'std':'std_flow',
        'sum':'total_flow',
        'count':'n_days',
        'max':'max_flow'
    })
    .reset_index()
)

In [61]:
# active_days = count non-zero days in last 90 days (approx)
recent = full_daily[full_daily['date'] >= (full_daily['date'].max() - pd.Timedelta(days=90))]
recent_agg = (
    recent.groupby('cust_id')['net_flow']
    .apply(lambda x: np.sum(x!=0)).reset_index().rename(columns={'net_flow':'active_days_90'})
)

In [62]:
agg = agg.merge(recent_agg, on='cust_id', how='left').fillna(0)

In [63]:
full_daily = full_daily.merge(agg, on="cust_id", how="left")

In [71]:
full_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676261 entries, 0 to 676260
Data columns (total 50 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date                 676261 non-null  datetime64[ns]
 1   cust_id              676261 non-null  object        
 2   net_flow             676261 non-null  float64       
 3   target_7d            676261 non-null  float64       
 4   roll_sum_7           676261 non-null  float64       
 5   roll_mean_7          676261 non-null  float64       
 6   roll_std_7           676261 non-null  float64       
 7   roll_count_nz_7      676261 non-null  float64       
 8   roll_sum_14          676261 non-null  float64       
 9   roll_mean_14         676261 non-null  float64       
 10  roll_std_14          676261 non-null  float64       
 11  roll_count_nz_14     676261 non-null  float64       
 12  roll_sum_30          676261 non-null  float64       
 13  roll_mean_30  

In [72]:
agg = full_daily.copy()

In [65]:
# merge static customer data
cust_static = df_customer.copy()
if 'BRTH_DT' in cust_static.columns:
    cust_static['BRTH_DT'] = pd.to_datetime(cust_static['BRTH_DT'], errors='coerce')
    latest_date = full_daily['date'].max()
    cust_static['age'] = ((latest_date - cust_static['BRTH_DT']).dt.days // 365).fillna(-1).astype(int)
else:
    cust_static['age'] = -1

In [66]:
keep_cols = ['cust_id','customer_type','parent_company_flag','CUST_GRP_ID','language','age']
keep_cols = [c for c in keep_cols if c in cust_static.columns]
cust_static_small = cust_static[keep_cols]
agg = agg.merge(cust_static_small, on='cust_id', how='left')

In [67]:
agg.head()

,date,cust_id,net_flow,target_7d,roll_sum_7,roll_mean_7,roll_std_7,roll_count_nz_7,roll_sum_14,roll_mean_14,...,std_flow,total_flow,n_days,max_flow,active_days_90,customer_type_y,parent_company_flag_y,CUST_GRP_ID_y,language_y,age_y
0,2023-01-01,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,-0.020048,2.00481,-0.020048,-0.020048,0.000000,1.0,-0.020048,-0.020048,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
1,2023-01-02,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.010024,0.014176,1.0,-0.020048,-0.010024,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
2,2023-01-03,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.006683,0.011575,1.0,-0.020048,-0.006683,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
3,2023-01-04,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.005012,0.010024,1.0,-0.020048,-0.005012,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1
4,2023-01-05,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,0.000000,2.00481,-0.020048,-0.004010,0.008966,1.0,-0.020048,-0.004010,...,1701.5551,-8333.792768,715,14033.666764,49.0,SME,1,NaN,RUS,-1


In [68]:
agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676261 entries, 0 to 676260
Data columns (total 55 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   date                   676261 non-null  datetime64[ns]
 1   cust_id                676261 non-null  object        
 2   net_flow               676261 non-null  float64       
 3   target_7d              676261 non-null  float64       
 4   roll_sum_7             676261 non-null  float64       
 5   roll_mean_7            676261 non-null  float64       
 6   roll_std_7             676261 non-null  float64       
 7   roll_count_nz_7        676261 non-null  float64       
 8   roll_sum_14            676261 non-null  float64       
 9   roll_mean_14           676261 non-null  float64       
 10  roll_std_14            676261 non-null  float64       
 11  roll_count_nz_14       676261 non-null  float64       
 12  roll_sum_30            676261 non-null  floa

In [75]:
agg.describe(include='all')

,date,cust_id,net_flow,target_7d,roll_sum_7,roll_mean_7,roll_std_7,roll_count_nz_7,roll_sum_14,roll_mean_14,...,parent_company_flag,CUST_GRP_ID,language,age,mean_flow,std_flow,total_flow,n_days,max_flow,active_days_90
count,676261,676261,6.762610e+05,6.762610e+05,6.762610e+05,6.762610e+05,6.762610e+05,676261.000000,6.762610e+05,6.762610e+05,...,676261.000000,229063,676261,676261.0,676261.000000,676261.000000,6.762610e+05,676261.000000,6.762610e+05,676261.00000
unique,NaN,958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,320,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,fedd8ec9267c5e537ccd33cc9aae6690be53e336df5423...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,168427e00053657623db6be4f2b1a4d7a590c6a893de10...,EST,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1440,551346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2023-12-23 02:00:26.052662784,NaN,4.878278e+00,6.505535e+01,2.082151e+01,-4.651937e+00,4.841248e+03,2.750799,4.465067e+01,-8.473188e+00,...,0.675752,NaN,NaN,-1.0,4.878278,8171.506804,8.313616e+03,710.912087,6.948477e+04,33.72226
min,2023-01-01 00:00:00,NaN,-7.808635e+06,-7.951560e+06,-7.951560e+06,-3.612219e+06,0.000000e+00,0.000000,-8.012290e+06,-3.612219e+06,...,0.000000,NaN,NaN,-1.0,-36268.970038,0.289466,-7.652753e+06,121.000000,1.967499e+00,0.00000
25%,2023-06-27 00:00:00,NaN,0.000000e+00,-4.923915e+02,-4.858005e+02,-6.962700e+01,1.111960e+01,1.000000,-1.012305e+03,-7.299385e+01,...,0.000000,NaN,NaN,-1.0,-9.213887,406.330439,-6.590077e+03,713.000000,3.591905e+03,12.00000
50%,2023-12-21 00:00:00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.729095e+02,2.000000,0.000000e+00,0.000000e+00,...,1.000000,NaN,NaN,-1.0,-0.015735,1317.508331,-8.474336e+00,723.000000,1.136912e+04,29.00000
75%,2024-06-18 00:00:00,NaN,0.000000e+00,6.561621e+02,6.546540e+02,9.382373e+01,1.602766e+03,4.000000,1.141780e+03,8.240566e+01,...,1.000000,NaN,NaN,-1.0,12.895044,4986.364903,9.336012e+03,724.000000,4.425077e+04,50.00000
max,2024-12-24 00:00:00,NaN,7.803744e+06,8.022838e+06,8.022838e+06,1.146120e+06,4.508648e+06,7.000000,5.630463e+06,4.021759e+05,...,1.000000,NaN,NaN,-1.0,4712.515047,475690.948747,3.411861e+06,724.000000,7.803744e+06,91.00000


## V. Customer Clustering

In [76]:
cluster_features = ['mean_flow','std_flow','total_flow','n_days','max_flow','active_days_90','roll_sum_7','roll_mean_7','roll_std_7','roll_count_nz_7','roll_sum_14','roll_mean_14','roll_std_14','roll_count_nz_14','roll_sum_30','roll_mean_30','roll_std_30','roll_count_nz_30','roll_sum_90','roll_mean_90','roll_std_90','roll_count_nz_90']
cluster_features = [c for c in cluster_features if c in agg.columns]
X_cluster = agg[cluster_features].fillna(0).copy()

In [77]:
# Log-transform skewed features to reduce dominance of whales
skewed = ['total_flow','max_flow','mean_flow']
for col in skewed:
    if col in X_cluster.columns:
        X_cluster[col] = np.sign(X_cluster[col]) * np.log1p(np.abs(X_cluster[col]))

In [78]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_cluster)

In [79]:
# Separate extreme whales defined as top 1% max_flow
whale_threshold = np.percentile(agg['max_flow'], 99)
whale_mask = agg['max_flow'] > whale_threshold
agg['is_whale'] = whale_mask

In [80]:
X_non_whale = X_scaled[~whale_mask.values]

In [88]:
n_clusters = 5  # adjust based on silhouette
gmm = GaussianMixture(n_components=n_clusters, random_state=42)
gmm_labels = gmm.fit_predict(X_non_whale)

In [89]:
agg['cluster'] = -1  # -1 for whales temporarily

# Assign non-whale clusters
agg.loc[~whale_mask, 'cluster'] = gmm_labels

# Assign whale cluster ID (e.g., max cluster + 1)
whale_cluster_id = agg['cluster'].max() + 1
agg.loc[whale_mask, 'cluster'] = whale_cluster_id

print(f"Whale cluster ID: {whale_cluster_id}, size: {whale_mask.sum()}")

Whale cluster ID: 5, size: 6465


In [90]:
sil_score = silhouette_score(X_non_whale, gmm_labels)
print(f"Silhouette score (non-whales): {sil_score:.4f}")

KeyboardInterrupt: 

In [ ]:
cluster_sizes = agg.groupby('cluster')['cust_id'].count()
print("Cluster sizes:\n", cluster_sizes)

In [109]:
cluster_summary = agg.groupby('cluster')[cluster_features + ['is_whale']].median()
print(cluster_summary)

          mean_flow       std_flow     total_flow  n_days      max_flow  \
cluster                                                                   
0         -0.013027     802.667719      -9.148030   721.0  6.903841e+03   
1         -4.286018   14010.968593   -3085.933044   724.0  1.135379e+05   
2          0.352151     318.608116     196.137000   481.0  2.963639e+03   
3        521.101733  127477.240859  377277.654729   723.0  1.062702e+06   

         active_days_90  is_whale  
cluster                            
0                  25.0       0.0  
1                  57.0       0.0  
2                   0.0       0.0  
3                  55.0       1.0  


In [111]:
# Merge cluster label back to daily dataframe
full_daily = full_daily.merge(agg[['cust_id','cluster']], on='cust_id', how='left')

In [112]:
full_daily.describe(include='all')

,date,cust_id,net_flow,target_7d,roll_sum_7,roll_mean_7,roll_std_7,roll_count_nz_7,roll_sum_14,roll_mean_14,...,roll_std_90,roll_count_nz_90,lag_1,lag_7_sum,dayofweek,is_weekend,day,month,year,cluster
count,676261,676261,6.762610e+05,6.762610e+05,6.762610e+05,6.762610e+05,6.762610e+05,676261.000000,6.762610e+05,6.762610e+05,...,6.762610e+05,676261.000000,6.762610e+05,6.762610e+05,676261.000000,676261.000000,676261.000000,676261.000000,676261.000000,676261.000000
unique,NaN,958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,fedd8ec9267c5e537ccd33cc9aae6690be53e336df5423...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2023-12-23 02:00:26.052662784,NaN,4.878278e+00,6.505535e+01,2.082151e+01,-4.651937e+00,4.841248e+03,2.750799,4.465067e+01,-8.473188e+00,...,6.789758e+03,33.324829,3.220474e+00,1.888568e+01,2.998409,0.285847,15.615376,6.409474,2023.485848,0.333252
min,2023-01-01 00:00:00,NaN,-7.808635e+06,-7.951560e+06,-7.951560e+06,-3.612219e+06,0.000000e+00,0.000000,-8.012290e+06,-3.612219e+06,...,0.000000e+00,0.000000,-7.808635e+06,-7.951560e+06,0.000000,0.000000,1.000000,1.000000,2023.000000,0.000000
25%,2023-06-27 00:00:00,NaN,0.000000e+00,-4.923915e+02,-4.858005e+02,-6.962700e+01,1.111960e+01,1.000000,-1.012305e+03,-7.299385e+01,...,2.286846e+02,14.000000,0.000000e+00,-4.824468e+02,1.000000,0.000000,8.000000,3.000000,2023.000000,0.000000
50%,2023-12-21 00:00:00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.729095e+02,2.000000,0.000000e+00,0.000000e+00,...,8.596845e+02,28.000000,0.000000e+00,0.000000e+00,3.000000,0.000000,16.000000,6.000000,2023.000000,0.000000
75%,2024-06-18 00:00:00,NaN,0.000000e+00,6.561621e+02,6.546540e+02,9.382373e+01,1.602766e+03,4.000000,1.141780e+03,8.240566e+01,...,3.481428e+03,49.000000,0.000000e+00,6.514894e+02,5.000000,1.000000,23.000000,9.000000,2024.000000,1.000000
max,2024-12-24 00:00:00,NaN,7.803744e+06,8.022838e+06,8.022838e+06,1.146120e+06,4.508648e+06,7.000000,5.630463e+06,4.021759e+05,...,2.554225e+06,90.000000,7.803744e+06,8.022838e+06,6.000000,1.000000,31.000000,12.000000,2024.000000,3.000000


## VI. Prepare modeling dataframe and split

In [113]:
# features to use
feature_cols = [
    'net_flow', 'roll_sum_7','roll_sum_14','roll_sum_30','roll_sum_90',
    'roll_mean_7','roll_mean_14','roll_mean_30','roll_mean_90',
    'roll_std_7','roll_std_14','roll_std_30','roll_std_90',
    'roll_count_nz_7','roll_count_nz_14','roll_count_nz_30','roll_count_nz_90',
    'lag_1','lag_7_sum','dayofweek','is_weekend','day','month','year','age'
]
feature_cols = [c for c in feature_cols if c in full_daily.columns]

In [114]:
# merge also some static cats
for c in ['customer_type','parent_company_flag','CUST_GRP_ID','language']:
    if c in cust_static_small.columns:
        full_daily[c] = full_daily['cust_id'].map(cust_static_small.set_index('cust_id')[c])
        if full_daily[c].dtype == object or full_daily[c].dtype.name=='category':
            full_daily[c] = full_daily[c].astype('category').cat.codes.fillna(-1).astype(int)
        else:
            full_daily[c] = pd.to_numeric(full_daily[c], errors='coerce').fillna(-1).astype(int)
        feature_cols.append(c)

In [115]:
# drop rows with too short history (optionally)
cust_history = full_daily.groupby('cust_id')['date'].nunique().reset_index().rename(columns={'date':'history_days'})
full_daily = full_daily.merge(cust_history,on='cust_id',how='left')
full_daily = full_daily[full_daily['history_days'] >= MIN_HISTORY_DAYS]

In [116]:
# split by date globally (chronological)
full_daily = full_daily.sort_values('date')
train_end = full_daily['date'].quantile(0.70)
valid_end = full_daily['date'].quantile(0.85)

In [117]:
train_df = full_daily[full_daily['date'] <= train_end]
valid_df = full_daily[(full_daily['date'] > train_end) & (full_daily['date'] <= valid_end)]
test_df = full_daily[full_daily['date'] > valid_end]

In [118]:
print('Train/Valid/Test shapes:', train_df.shape, valid_df.shape, test_df.shape)

Train/Valid/Test shapes: (473873, 33) (101394, 33) (100994, 33)


## VII. Modeling Helpers

In [119]:
def make_dmatrix(X, y=None):
    if y is None:
        return xgb.DMatrix(X)
    return xgb.DMatrix(X, label=y)

In [120]:
def safe_mape(y_true, y_pred):
    eps = 1e-6
    return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + eps))) * 100

In [122]:
# function to run optuna per-cluster
def tune_and_train_cluster(cluster_id, train_df, valid_df, test_df, feature_cols, n_trials=30, cap_pct=GLOBAL_CAP_PCT):
    print(f"\n--- Cluster {cluster_id} tuning & training ---")
    # filter by cluster
    tr = train_df[train_df['cluster']==cluster_id]
    va = valid_df[valid_df['cluster']==cluster_id]
    te = test_df[test_df['cluster']==cluster_id]
    
    if len(tr) < 1000 or tr['cust_id'].nunique() < 10:
        print('Cluster too small for tuning (falling back to global params).')
        return None
    
    X_tr = tr[feature_cols].fillna(0)
    X_va = va[feature_cols].fillna(0)
    X_te = te[feature_cols].fillna(0)
    
    y_tr = tr['target_7d'].values
    y_va = va['target_7d'].values
    y_te = te['target_7d'].values
    
    # cap training target to reduce tail influence
    cap = np.percentile(y_tr, cap_pct)
    y_tr_cap = np.clip(y_tr, -cap, cap)

    # log1p transform of positive part (we predict positive flows primarily)
    # To keep sign info, we can model positive and negative separately, but here we model signed values using signed log
    def sign_log1p(x):
        return np.sign(x) * np.log1p(np.abs(x))
    def inv_sign_log1p(y):
        return np.sign(y) * (np.expm1(np.abs(y)))
    
    y_tr_trans = sign_log1p(y_tr_cap)
    y_va_trans = sign_log1p(y_va)
    
    dtrain = make_dmatrix(X_tr, y_tr_trans)
    dvalid = make_dmatrix(X_va, y_va_trans)

    def objective(trial):
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': TREE_METHOD,
            'device': DEVICE,
            'eta': trial.suggest_float('eta', 0.01, 0.2, log=True),
            'max_depth': trial.suggest_int('max_depth', 4, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'alpha': trial.suggest_float('alpha', 0.0, 10.0),
            'lambda': trial.suggest_float('lambda', 0.0, 10.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 50)
        }
        evals = [(dtrain, 'train'), (dvalid, 'valid')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=2000,
            evals=evals,
            early_stopping_rounds=50,
            verbose_eval=False
        )
        pred_va = bst.predict(dvalid)
        # inverse transform
        pred_va_inv = inv_sign_log1p(pred_va)
        rmse = mean_squared_error(y_va, pred_va_inv) ** 0.5
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    
    print('Best RMSE on validation (from objective):', study.best_value)
    print('Best params:', study.best_params)
    
    # Train final model on combined train+valid
    combo_X = pd.concat([X_tr, X_va], axis=0)
    combo_y = np.concatenate([y_tr_trans, y_va_trans], axis=0)
    dcombo = make_dmatrix(combo_X, combo_y)

    best_params = study.best_params
    params_final = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'tree_method': TREE_METHOD,
        'device': DEVICE,
        'eta': best_params['eta'],
        'max_depth': best_params['max_depth'],
        'subsample': best_params['subsample'],
        'colsample_bytree': best_params['colsample_bytree'],
        'alpha': best_params['alpha'],
        'lambda': best_params['lambda'],
        'min_child_weight': best_params['min_child_weight']
    }
    
    
    evals_final = [(dcombo, 'train'), (dvalid, 'valid')]
    bst_final = xgb.train(
        params_final,
        dcombo,
        num_boost_round=2000,
        evals=evals_final,
        early_stopping_rounds=50,
        verbose_eval=50
    )

    # Evaluate on test
    dtest = make_dmatrix(X_te)
    pred_te = bst_final.predict(dtest)
    pred_te_inv = inv_sign_log1p(pred_te)
    
    rmse_test = mean_squared_error(y_te, pred_te_inv) ** 0.5
    mae_test = mean_absolute_error(y_te, pred_te_inv)
    mape_test = safe_mape(y_te, pred_te_inv)
    
    print(f"Cluster {cluster_id} TEST RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}, MAPE: {mape_test:.2f}%")
    
    # Save model
    model_path = os.path.join(OUTPUT_DIR, f'xgb_cluster_{cluster_id}.model')
    bst_final.save_model(model_path)
    print('Saved model to', model_path)
    
    # return performance and model
    return {
        'cluster': cluster_id,
        'model': bst_final,
        'rmse_test': rmse_test,
        'mae_test': mae_test,
        'mape_test': mape_test,
        'cap': cap
    }

In [123]:
# Separate model for clusters identified as whales
def whale_forecast(customer_df):
    df = customer_df.copy().sort_values("date")

    # Compute weekday
    df["weekday"] = df["date"].dt.weekday

    # 1) Seasonal Naive: mean flow on the same weekday for the last 8 weeks
    last_8_weeks = df[df["date"] >= df["date"].max() - pd.Timedelta(days=56)]
    seasonal_forecast = (
        last_8_weeks.groupby("weekday")["net_flow"]
        .mean()
        .reindex(range(7), fill_value=np.nan)
        .to_numpy()
    )

    # 2) Rolling median (30 days)
    median_30d = df[df["date"] >= df["date"].max() - pd.Timedelta(days=30)]["net_flow"].median()

    # 3) Last-7-day sum
    last7_sum = df[df["date"] >= df["date"].max() - pd.Timedelta(days=7)]["net_flow"].sum()

    # Build the 7-day forecast
    forecast = []

    for i in range(7):
        val = seasonal_forecast[i]

        # Fallbacks
        if np.isnan(val):
            if not np.isnan(median_30d):
                val = median_30d
            else:
                val = last7_sum / 7.0

        forecast.append(val)

    return np.array(forecast)

## VIII. Training per-cluster models

In [125]:
results = []
all_predictions = []  # to store per-row predictions

for c in sorted(agg['cluster'].unique()):
    # subset test data for this cluster
    te = test_df[test_df['cluster'] == c]

    if len(te) == 0:
        continue  # skip empty clusters

    if c == 3:
        print(f"Cluster {c} is marked as whale cluster. Using whale_forecast for predictions.")

        # loop per customer in this cluster
        for cust_id, group in te.groupby('cust_id'):
            cust_history = daily_cashflow[daily_cashflow["cust_id"] == cust_id]
            preds = whale_forecast(cust_history)

            tmp = group.copy()
            tmp['y_pred'] = preds  # predicted 7-day flow
            all_predictions.append(tmp)
    
    else:
        # ---- use XGBoost / tuned model ----
        res = tune_and_train_cluster(c, train_df, valid_df, test_df, feature_cols,
                                     n_trials=OPTUNA_TRIALS, cap_pct=GLOBAL_CAP_PCT)

        # store per-row predictions
        dtest = xgb.DMatrix(te[feature_cols])
        te_preds = res['model'].predict(dtest)
        tmp = te.copy()
        tmp['y_pred'] = te_preds
        all_predictions.append(tmp)

        results.append(res)

# Combine all predictions into a single dataframe
pred_df = pd.concat(all_predictions, ignore_index=True)


--- Cluster 0 tuning & training ---


[I 2025-12-10 01:27:02,821] A new study created in memory with name: no-name-6c8099cd-1fc0-4b5d-bfab-ce6113064801
[I 2025-12-10 01:27:05,355] Trial 0 finished with value: 4061.823134869642 and parameters: {'eta': 0.07115584722422022, 'max_depth': 8, 'subsample': 0.666911675143514, 'colsample_bytree': 0.8799101733515802, 'alpha': 2.9799285333608183, 'lambda': 4.974638239690661, 'min_child_weight': 2}. Best is trial 0 with value: 4061.823134869642.
[I 2025-12-10 01:27:11,468] Trial 1 finished with value: 4047.6763273579695 and parameters: {'eta': 0.022232470093406422, 'max_depth': 8, 'subsample': 0.8359943542654238, 'colsample_bytree': 0.6950511913160214, 'alpha': 0.8575364369261174, 'lambda': 3.71044503553706, 'min_child_weight': 26}. Best is trial 1 with value: 4047.6763273579695.
[I 2025-12-10 01:27:12,869] Trial 2 finished with value: 4043.686936087758 and parameters: {'eta': 0.13151683849940707, 'max_depth': 6, 'subsample': 0.9559320231287027, 'colsample_bytree': 0.5843467379063764,

Best RMSE on validation (from objective): 4031.690328629405
Best params: {'eta': 0.05421843345864502, 'max_depth': 4, 'subsample': 0.8487943941483242, 'colsample_bytree': 0.5972057387142025, 'alpha': 7.540193818721859, 'lambda': 6.683615300264692, 'min_child_weight': 18}
[0]	train-rmse:5.72570	valid-rmse:5.61254
[50]	train-rmse:5.52382	valid-rmse:5.41700
[100]	train-rmse:5.47627	valid-rmse:5.37047
[150]	train-rmse:5.44262	valid-rmse:5.33849
[200]	train-rmse:5.41685	valid-rmse:5.31418
[250]	train-rmse:5.39496	valid-rmse:5.29334
[300]	train-rmse:5.37490	valid-rmse:5.27451
[350]	train-rmse:5.35782	valid-rmse:5.25845
[400]	train-rmse:5.34215	valid-rmse:5.24356
[450]	train-rmse:5.32835	valid-rmse:5.23012
[500]	train-rmse:5.31331	valid-rmse:5.21589
[550]	train-rmse:5.30146	valid-rmse:5.20471
[600]	train-rmse:5.28931	valid-rmse:5.19459
[650]	train-rmse:5.27836	valid-rmse:5.18521
[700]	train-rmse:5.26826	valid-rmse:5.17593
[750]	train-rmse:5.25787	valid-rmse:5.16648
[800]	train-rmse:5.24861	va

[I 2025-12-10 01:31:20,371] A new study created in memory with name: no-name-27cac670-a0dc-4e34-beae-72a3e14b5b27
[I 2025-12-10 01:31:21,198] Trial 0 finished with value: 66575.80063383201 and parameters: {'eta': 0.16448980824416462, 'max_depth': 8, 'subsample': 0.7690536462641017, 'colsample_bytree': 0.6147724009610342, 'alpha': 0.703621078951221, 'lambda': 7.195518645447986, 'min_child_weight': 19}. Best is trial 0 with value: 66575.80063383201.
[I 2025-12-10 01:31:26,535] Trial 1 finished with value: 197541.28380958765 and parameters: {'eta': 0.03009799214658767, 'max_depth': 4, 'subsample': 0.8295919275394722, 'colsample_bytree': 0.6673921794706253, 'alpha': 3.0686094975291747, 'lambda': 7.298623617939192, 'min_child_weight': 28}. Best is trial 0 with value: 66575.80063383201.
[I 2025-12-10 01:31:30,612] Trial 2 finished with value: 60584.11319080248 and parameters: {'eta': 0.04713452301906735, 'max_depth': 10, 'subsample': 0.6961021431907194, 'colsample_bytree': 0.9850501710267203

Best RMSE on validation (from objective): 60182.39356450035
Best params: {'eta': 0.010202466557726855, 'max_depth': 10, 'subsample': 0.8628067316230985, 'colsample_bytree': 0.6871685300285245, 'alpha': 3.6046298592301733, 'lambda': 1.7520136971902662, 'min_child_weight': 33}
[0]	train-rmse:8.68782	valid-rmse:8.53805
[50]	train-rmse:8.18022	valid-rmse:8.04394
[100]	train-rmse:7.84692	valid-rmse:7.71887
[150]	train-rmse:7.59410	valid-rmse:7.47044
[200]	train-rmse:7.39847	valid-rmse:7.27780
[250]	train-rmse:7.24289	valid-rmse:7.13071
[300]	train-rmse:7.11830	valid-rmse:7.00983
[350]	train-rmse:7.00488	valid-rmse:6.90119
[400]	train-rmse:6.91092	valid-rmse:6.81081
[450]	train-rmse:6.83301	valid-rmse:6.73871
[500]	train-rmse:6.75546	valid-rmse:6.66537
[550]	train-rmse:6.68040	valid-rmse:6.59442
[600]	train-rmse:6.61084	valid-rmse:6.53045
[650]	train-rmse:6.54733	valid-rmse:6.47254
[700]	train-rmse:6.48146	valid-rmse:6.41033
[750]	train-rmse:6.41856	valid-rmse:6.34828
[800]	train-rmse:6.3514

[I 2025-12-10 01:35:16,616] A new study created in memory with name: no-name-8b0cefe4-5841-4088-9a71-a805a7125bba
[I 2025-12-10 01:35:17,299] Trial 0 finished with value: 6155.4678677771 and parameters: {'eta': 0.032481386724490316, 'max_depth': 4, 'subsample': 0.6561773478513749, 'colsample_bytree': 0.8415949946641764, 'alpha': 3.2082355777101492, 'lambda': 1.1804099287995762, 'min_child_weight': 30}. Best is trial 0 with value: 6155.4678677771.
[I 2025-12-10 01:35:17,772] Trial 1 finished with value: 6155.095180904972 and parameters: {'eta': 0.03576924795743648, 'max_depth': 9, 'subsample': 0.6932737436350007, 'colsample_bytree': 0.8589558579673247, 'alpha': 9.133874774926724, 'lambda': 2.201156014770337, 'min_child_weight': 23}. Best is trial 1 with value: 6155.095180904972.
[I 2025-12-10 01:35:18,443] Trial 2 finished with value: 6157.382603724311 and parameters: {'eta': 0.014875439748907659, 'max_depth': 7, 'subsample': 0.9860662019520068, 'colsample_bytree': 0.655079489896568, 'a

Best RMSE on validation (from objective): 6151.82093274179
Best params: {'eta': 0.12575292421281856, 'max_depth': 9, 'subsample': 0.6242321191708966, 'colsample_bytree': 0.8296035136501949, 'alpha': 5.738604442807331, 'lambda': 2.9056638428907884, 'min_child_weight': 31}
[0]	train-rmse:4.57288	valid-rmse:3.06345
[50]	train-rmse:3.29335	valid-rmse:2.35367
[100]	train-rmse:2.76641	valid-rmse:2.05612
[150]	train-rmse:2.40506	valid-rmse:1.83208
[200]	train-rmse:2.16680	valid-rmse:1.68223
[250]	train-rmse:1.95438	valid-rmse:1.54180
[300]	train-rmse:1.79183	valid-rmse:1.43677
[350]	train-rmse:1.65013	valid-rmse:1.33326
[400]	train-rmse:1.53429	valid-rmse:1.25296
[450]	train-rmse:1.43273	valid-rmse:1.17704
[500]	train-rmse:1.34768	valid-rmse:1.10820
[550]	train-rmse:1.26477	valid-rmse:1.03906
[600]	train-rmse:1.19307	valid-rmse:0.97665
[650]	train-rmse:1.13225	valid-rmse:0.93387
[700]	train-rmse:1.07821	valid-rmse:0.88996
[750]	train-rmse:1.03105	valid-rmse:0.85389
[800]	train-rmse:0.98608	va

ValueError: Length of values (7) does not match length of index (114)

## IX. Global evaluation and combining cluster predictions

In [ ]:
preds = []
for c in sorted(agg['cluster'].unique()):
    te = test_df[test_df['cluster']==c]
    if te.empty:
        continue
    if c in whale_clusters:
        # baseline use
        p = te['lag_7_sum'].values
    else:
        # load model path
        model_path = os.path.join(OUTPUT_DIR, f'xgb_cluster_{c}.model')
        if not os.path.exists(model_path):
            print('Model missing for cluster', c)
            continue
        bst = xgb.Booster()
        bst.load_model(model_path)
        dte = xgb.DMatrix(te[feature_cols].fillna(0))
        p_raw = bst.predict(dte)
        # inverse transform
        p = np.sign(p_raw) * (np.expm1(np.abs(p_raw)))
    preds.append(pd.DataFrame({'cust_id': te['cust_id'].values, 'date': te['date'].values, 'y_true': te['target_7d'].values, 'y_pred': p}))

In [ ]:
pred_df = pd.concat(preds, ignore_index=True)

In [ ]:
global_rmse = math.sqrt(((pred_df['y_true'] - pred_df['y_pred'])**2).mean())
global_mae = (np.abs(pred_df['y_true'] - pred_df['y_pred']).mean())
global_mape = safe_mape(pred_df['y_true'].values, pred_df['y_pred'].values)

print(f"Global test RMSE: {global_rmse:.2f}, MAE: {global_mae:.2f}, MAPE: {global_mape:.2f}%")

In [ ]:
# Per-cluster RMSE
per_cluster = pred_df.groupby('cust_id').apply(lambda g: math.sqrt(((g['y_true']-g['y_pred'])**2).mean())).reset_index(name='rmse_cust')
per_cluster = per_cluster.merge(agg[['cust_id','cluster']], on='cust_id', how='left')
cluster_rmse = per_cluster.groupby('cluster')['rmse_cust'].median().reset_index().rename(columns={'rmse_cust':'median_rmse_per_customer'})
print('\nMedian per-customer RMSE by cluster:')
print(cluster_rmse)

In [ ]:
raw_cluster_rmse = []

for cluster_id, group in pred_df.merge(
        agg[['cust_id','cluster']], on='cust_id', how='left'
    ).groupby('cluster'):

    y_true = group['y_true'].values
    y_pred = group['y_pred'].values

    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    raw_cluster_rmse.append((cluster_id, rmse))

raw_cluster_rmse = (
    pd.DataFrame(raw_cluster_rmse, columns=['cluster', 'raw_cluster_rmse'])
)

print("\nRAW RMSE per cluster (all rows pooled):")
print(raw_cluster_rmse)